# Pinecone Assitant Getting Started

Welcome to the getting started Notebook for [Pinecone assistant](https://www.pinecone.io/blog/pinecone-assistant/)!

**!IMPORTANT!** - This code will not work unless your account has been invited to the private beta

In [ ]:
# Install pinecone client and assistants plugin
!pip install --upgrade pinecone-client pinecone-plugin-assistant

In [ ]:
# Install Pinecone notebook utilities including the Pinecone Connect widget
!pip install pinecone-notebooks==0.1.1

In [ ]:
import os

if not os.environ.get("PINECONE_API_KEY"):
    from pinecone_notebooks.colab import Authenticate
    Authenticate()

In [ ]:
from google.colab import userdata
from pinecone import Pinecone

pc = Pinecone(api_key=os.environ.get('PINECONE_API_KEY') or userdata.get('PINECONE_API_KEY'))

assistant_name = 'HelloPineconeAssistant'

metadata = {"author": "Jane Doe", "version": "1.0"}

assistant = pc.assistant.create_assistant(
    assistant_name=assistant_name,
    metadata=metadata,
    timeout=30 # Wait 30 seconds for model creation to complete.
)

In [ ]:
# Get assistant status
assistant_status = pc.assistant.describe_assistant(assistant_name=assistant_name)
assistant_status

In [ ]:
import random

# Function to generate a random toy title
def generate_toy_title():
    adjectives = ["Amazing", "Super", "Fantastic", "Incredible", "Awesome", "Magical", "Ultimate", "Deluxe"]
    nouns = ["Robot", "Doll", "Blocks", "Puzzle", "Car", "Train", "Spaceship", "Dinosaur", "Unicorn", "Dragon"]
    return f"{random.choice(adjectives)} {random.choice(nouns)}"

# Function to generate a random toy description
def generate_toy_description():
    features = [
        "Made from high-quality materials",
        "Encourages imaginative play",
        "Develops problem-solving skills",
        "Suitable for ages 3 and up",
        "Battery-operated with flashing lights",
        "Includes multiple accessories",
        "Educational and fun",
        "Easy to clean and store",
        "Durable and long-lasting",
        "Promotes hand-eye coordination"
    ]
    return "\n".join(random.sample(features, 4))

# Generate 75 toy entries
toy_entries = []
for i in range(75):
    title = generate_toy_title()
    description = generate_toy_description()
    entry = f"{title}\n\n{description}\n\n{'='*30}\n\n"
    toy_entries.append(entry)

# Write to a local file in Colab
file_path = "/content/toy_descriptions.txt"

with open(file_path, 'w') as f:
    f.writelines(toy_entries)

print(f"Created toy descriptions file at {file_path}")

# Display the first few entries as a sample
with open(file_path, 'r') as f:
    print(f.read().split('='*30)[:3])

In [ ]:
import os

# Target our existing Pinecone Assistant
assistant = pc.assistant.Assistant(
    assistant_name=assistant_name,
)

# Upload the file
if os.path.exists(file_path):
    response = assistant.upload_file(
        file_path=file_path,
        timeout=None
    )
    print(f"Uploaded {file_path}")
    print("Response:", response)
else:
    print(f"File not found: {file_path}")

In [ ]:
# List uploaded files our assistant is aware of
files = assistant.list_files()
files

In [ ]:
# Chat with your Pinecone assistant, which automatically references
# your uploaded documents in its responses
from pinecone_plugins.assistant.models.chat import Message
chat_context = [Message(content='Which toys include Unicorn in their name?')]
response = assistant.chat_completions(messages=chat_context)
print(response)